In [48]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 메모리 사용량 감소 함수
from my_def3 import reduce_memory_usage

# 분기별 파일 선택 함수
def select_csv_file(shift):
    """
    주어진 분기 이동에 따라 적절한 CSV 파일을 선택합니다.

    Parameters:
        shift (int): 몇 분기 뒤를 예측할 것인지 (1, 2, 3, 4)

    Returns:
        str: 선택된 CSV 파일 경로
    """
    file_mapping = {
        1: 'D:\project\Project 2\머신러닝 테스트\Merged_Sales_data_v2.6(shift1).csv',
        2: 'D:\project\Project 2\머신러닝 테스트\Merged_Sales_data_v2.6(shift2).csv',
        3: 'D:\project\Project 2\머신러닝 테스트\Merged_Sales_data_v2.6(shift3).csv',
        4: 'D:\project\Project 2\머신러닝 테스트\Merged_Sales_data_v2.6(shift4).csv'
    }
    return file_mapping.get(shift, None)

<>:16: SyntaxWarning: invalid escape sequence '\p'
<>:17: SyntaxWarning: invalid escape sequence '\p'
<>:18: SyntaxWarning: invalid escape sequence '\p'
<>:19: SyntaxWarning: invalid escape sequence '\p'
<>:16: SyntaxWarning: invalid escape sequence '\p'
<>:17: SyntaxWarning: invalid escape sequence '\p'
<>:18: SyntaxWarning: invalid escape sequence '\p'
<>:19: SyntaxWarning: invalid escape sequence '\p'
C:\Users\human\AppData\Local\Temp\ipykernel_984\446977432.py:16: SyntaxWarning: invalid escape sequence '\p'
  1: 'D:\project\Project 2\머신러닝 테스트\Merged_Sales_data_v2.6(shift1).csv',
C:\Users\human\AppData\Local\Temp\ipykernel_984\446977432.py:17: SyntaxWarning: invalid escape sequence '\p'
  2: 'D:\project\Project 2\머신러닝 테스트\Merged_Sales_data_v2.6(shift2).csv',
C:\Users\human\AppData\Local\Temp\ipykernel_984\446977432.py:18: SyntaxWarning: invalid escape sequence '\p'
  3: 'D:\project\Project 2\머신러닝 테스트\Merged_Sales_data_v2.6(shift3).csv',
C:\Users\human\AppData\Local\Temp\ipykernel_98

In [50]:
# 데이터 불러오기 및 메모리 사용량 최적화
def load_and_optimize_data(shift):
    file_path = select_csv_file(shift)
    if file_path is None:
        raise ValueError(f"지원되지 않는 shift 값입니다: {shift}")
    df = pd.read_csv(file_path)
    df = reduce_memory_usage(df)
    return df

In [ ]:
# 입력 변수 설정
def train_sales_model(category, shift):
    """
    랜덤 포레스트 모델을 사용하여 주어진 업종별 카테고리를 기반으로 매출 예측 모델을 학습합니다.

    Parameters:
        category (str): 업종별 카테고리
        shift (int): 몇 분기 뒤를 예측할 것인지 (예: 1, 2, 3, 4)

    Returns:
        model: 학습된 랜덤 포레스트 모델
        pd.DataFrame: 학습에 사용된 데이터프레임 (피처)
    """
    # 선택 가능한 업종별 카테고리 목록
    valid_categories = ['생활서비스', '소매_편의', '식음료', '의료_건강', '가구_전자기기', '교육_학원', '미용_패션', '취미_레저', '부동산_숙박']
    if category not in valid_categories:
        raise ValueError(f"지원되지 않는 업종별 카테고리입니다: {category}. 선택 가능한 카테고리: {valid_categories}")

    # 데이터 불러오기
    df = load_and_optimize_data(shift)

    # 해당 카테고리 데이터 필터링
    df_filtered = df[df['업종별카테고리'] == category].copy()
    if df_filtered.empty:
        raise ValueError("해당 업종별 카테고리에 대한 데이터가 없습니다.")
    
    # 행정동 데이터 저장
    original_dong = df_filtered['행정동']

    # 원핫인코딩 수행
    df_filtered = pd.get_dummies(df_filtered, columns=['행정동', '업종별카테고리'], drop_first=True)

    # 피처와 타겟 설정
    features = ['년분기', '인구수', '지역생활인구', '장기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + \
               [col for col in df_filtered.columns if col.startswith('행정동_') or col.startswith('업종별카테고리_')]
    target = '월매출(점포)'

    # 피처와 타겟 데이터 분리
    X = df_filtered[features]
    y = df_filtered[target]

    # 스케일링 (Standard Scaler)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # 랜덤 포레스트 회귀 모델 학습 (최종 모델)
    model = RandomForestRegressor(n_estimators=500, max_depth=50, random_state=42)
    model.fit(X, y)

    return model, X, original_dong

In [57]:
def predict_sales(model, X, original_dong):
    """
    학습된 모델을 사용하여 매출을 예측합니다.

    Parameters:
        model: 학습된 랜덤 포레스트 모델
        X (pd.DataFrame): 예측에 사용할 피처 데이터프레임

    Returns:
        pd.DataFrame: 예측 결과 데이터프레임
    """
    # 예측 수행
    y_pred = model.predict(X)

    # 예측 결과를 데이터프레임으로 결합
    df_results = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    df_results['예측_매출'] = y_pred
    df_results['행정동'] = original_dong.values

    # 예측 매출 상위 5개 행정동 추출
    top5_dongs = df_results.groupby('행정동')['예측_매출'].mean().sort_values(ascending=False).head(5).reset_index()

    return top5_dongs

In [58]:
category = '부동산_숙박'
shift = 3
model, X, original_dong = train_sales_model(category, shift)
result = predict_sales(model, X, original_dong)
print(result)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62298 entries, 0 to 62297
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년분기         62298 non-null  int16  
 1   행정동         62298 non-null  object 
 2   업종별카테고리     62298 non-null  object 
 3   월매출(점포)     62298 non-null  int64  
 4   인구수         62298 non-null  float16
 5   지역생활인구      62298 non-null  float32
 6   장기외국인       62298 non-null  float32
 7   단기외국인       62298 non-null  float32
 8   주차장면적(면)    62298 non-null  float32
 9   주차장개수(개소)   62298 non-null  float16
 10  학교수         62298 non-null  float16
 11  학생수         62298 non-null  float16
 12  버스정류장수      62298 non-null  float16
 13  년분기_미룸      62298 non-null  float16
 14  월매출_shift3  62298 non-null  float32
dtypes: float16(6), float32(5), int16(1), int64(1), object(2)
memory usage: 3.4+ MB
None
     행정동         예측_매출
0    신림동  1.486674e+08
1   상도1동  4.422469e+07
2     길동  3.358851e+07
3  노량진1동  2.7

In [60]:
category = '부동산_숙박'
shift = 2
model, X, original_dong = train_sales_model(category, shift)
result = predict_sales(model, X, original_dong)
print(result)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65647 entries, 0 to 65646
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년분기         65647 non-null  int16  
 1   행정동         65647 non-null  object 
 2   업종별카테고리     65647 non-null  object 
 3   월매출(점포)     65647 non-null  int64  
 4   인구수         65647 non-null  float16
 5   지역생활인구      65647 non-null  float32
 6   장기외국인       65647 non-null  float32
 7   단기외국인       65647 non-null  float32
 8   주차장면적(면)    65647 non-null  float32
 9   주차장개수(개소)   65647 non-null  float16
 10  학교수         65647 non-null  float16
 11  학생수         65647 non-null  float16
 12  버스정류장수      65647 non-null  float16
 13  년분기_미룸      65647 non-null  float16
 14  월매출_shift2  65647 non-null  float32
dtypes: float16(6), float32(5), int16(1), int64(1), object(2)
memory usage: 3.6+ MB
None
     행정동         예측_매출
0    신림동  1.503405e+08
1   상도1동  4.112848e+07
2     길동  3.488948e+07
3   장안2동  2.9

In [ ]:
category = '한식'
shift = 3
model, X = train_sales_model(category, shift)
result = predict_sales(model, X)
print(result)

ValueError: 지원되지 않는 업종별 카테고리입니다: 한식. 선택 가능한 카테고리: ['생활서비스', '소매_편의', '식음료', '의료_건강', '가구_전자기기', '교육_학원', '미용_패션', '취미_레저', '부동산_숙박']